In [1]:
import pandas as pd;
import numpy as np;
import statsmodels.api as sm;
import statsmodels.discrete.discrete_model as smdiscrete
pd.set_option('use_inf_as_na', True)

In [2]:
meta_df = pd.read_csv("stockmetadata.csv")
fdata_df = pd.read_csv("corpfund.csv")
fdata_df = fdata_df[fdata_df['dimension']=='ARQ']
fdata_df['datekey'] = pd.to_datetime(fdata_df['datekey'])
df_left = pd.merge(fdata_df, meta_df, on='ticker', how='left')
df_left = df_left.set_index('datekey')

In [3]:
industrydummies = pd.get_dummies(df_left['sicsector'])
industrydummies.sum()		#purely for exploring the data, has no other purpose
industrydummies.describe()      #purely for exploring the data, has no other purpose

,Agriculture Forestry And Fishing,Construction,Finance Insurance And Real Estate,Manufacturing,Mining,Retail Trade,Services,Transportation Communications Electric Gas And Sanitary Service,Wholesale Trade
count,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000
mean,0.002713,0.009157,0.176068,0.296688,0.036425,0.036811,0.114986,0.052585,0.019220
std,0.052019,0.095253,0.380879,0.456799,0.187346,0.188297,0.319006,0.223203,0.137297
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
data_w_dummies = pd.concat([df_left,industrydummies], axis=1)

data_w_dummies.drop(['Wholesale Trade'], inplace=True, axis=1)	#drop 1 dummy variable
data_w_dummies['epratio'] = data_w_dummies['eps']/data_w_dummies['price']	#generate dependent variable
data_w_dummies['operatingmargin'] = data_w_dummies['opinc'] / data_w_dummies['revenue']		#generate independent variable

In [5]:

#initial analysis
result = sm.OLS(data_w_dummies['epratio'], sm.add_constant(data_w_dummies[['operatingmargin']]), missing='drop').fit()
result.summary()
result = sm.OLS(data_w_dummies['epratio'], sm.add_constant(data_w_dummies[['operatingmargin', 'Agriculture Forestry And Fishing', 'Construction','Finance Insurance And Real Estate', 'Manufacturing', 'Mining','Retail Trade', 'Services','Transportation Communications Electric Gas And Sanitary Service']]), missing='drop').fit()
result.summary()
data_w_dummies['lnoperatingmargin'] = np.log(data_w_dummies['operatingmargin'])
result = sm.OLS(data_w_dummies['epratio'], sm.add_constant(data_w_dummies[['lnoperatingmargin']]), missing='drop').fit()
result.summary()
data_w_dummies['lnepratio'] = np.log(data_w_dummies['epratio'])
result = sm.OLS(data_w_dummies['lnepratio'], sm.add_constant(data_w_dummies[['lnoperatingmargin']]), missing='drop').fit()
result.summary()

C:\Users\leela\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\leela\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\leela\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              lnepratio   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     8846.
Date:                Sun, 11 Oct 2020   Prob (F-statistic):               0.00
Time:                        08:46:11   Log-Likelihood:            -1.5681e+05
No. Observations:              115861   AIC:                         3.136e+05
Df Residuals:                  115859   BIC:                         3.136e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -3.7721      0.006   -602.657      0.000      -3.784      -3.760
lnoperatingmargin     0.2638      0.003     94.054      0.000       0.258       0.269
==============================================================================
Omnibus:                    18365.631   Durbin-Watson:                   1.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           158342.851
Skew:                           0.510   Prob(JB):                         0.00
Kurtosis:                       8.635   Cond. No.                         5.93
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
#with dummy variables
result = sm.OLS(data_w_dummies['lnepratio'], sm.add_constant(data_w_dummies[['lnoperatingmargin', 'Agriculture Forestry And Fishing', 'Construction','Finance Insurance And Real Estate', 'Manufacturing', 'Mining','Retail Trade', 'Services','Transportation Communications Electric Gas And Sanitary Service']]), missing='drop').fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              lnepratio   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     1188.
Date:                Sun, 11 Oct 2020   Prob (F-statistic):               0.00
Time:                        08:46:35   Log-Likelihood:            -1.5595e+05
No. Observations:              115861   AIC:                         3.119e+05
Df Residuals:                  115851   BIC:                         3.120e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===================================================================================================================================
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                              -3.6332      0.009   -419.187      0.000      -3.650      -3.616
lnoperatingmargin                                                   0.2969      0.003     96.466      0.000       0.291       0.303
Agriculture Forestry And Fishing                                    0.6587      0.058     11.417      0.000       0.546       0.772
Construction                                                        0.3404      0.025     13.352      0.000       0.290       0.390
Finance Insurance And Real Estate                                  -0.1784      0.008    -21.688      0.000      -0.195      -0.162
Manufacturing                                                      -0.0182      0.008     -2.360      0.018      -0.033      -0.003
Mining                                                             -0.0450      0.018     -2.489      0.013      -0.080      -0.010
Retail Trade                                                        0.1256      0.014      9.187      0.000       0.099       0.152
Services                                                           -0.2630      0.010    -26.005      0.000      -0.283      -0.243
Transportation Communications Electric Gas And Sanitary Service    -0.1285      0.012    -10.640      0.000      -0.152      -0.105
==============================================================================
Omnibus:                    18757.262   Durbin-Watson:                   1.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           159948.347
Skew:                           0.533   Prob(JB):                         0.00
Kurtosis:                       8.657   Cond. No.                         51.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
#clustering standard variables
data_w_dummies.dropna(subset = ['lnepratio', 'lnoperatingmargin'], inplace=True)	#because of a bug in python where fillna is not working perfectly
#note that we can cannot cluster by str variables in python, hence using siccode instead of sicsector
result = sm.OLS(data_w_dummies['lnepratio'], sm.add_constant(data_w_dummies[['lnoperatingmargin', 'Agriculture Forestry And Fishing', 'Construction','Finance Insurance And Real Estate', 'Manufacturing', 'Mining','Retail Trade', 'Services','Transportation Communications Electric Gas And Sanitary Service']]), missing='drop').fit(cov_type='cluster', cov_kwds={'groups': data_w_dummies['siccode']})
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              lnepratio   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     44.65
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           2.30e-80
Time:                        08:46:51   Log-Likelihood:            -1.5595e+05
No. Observations:              115861   AIC:                         3.119e+05
Df Residuals:                  115851   BIC:                         3.120e+05
Df Model:                           9                                         
Covariance Type:              cluster                                         
===================================================================================================================================
                                                                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                              -3.6332      0.034   -107.420      0.000      -3.700      -3.567
lnoperatingmargin                                                   0.2969      0.016     18.678      0.000       0.266       0.328
Agriculture Forestry And Fishing                                    0.6587      0.357      1.847      0.065      -0.040       1.358
Construction                                                        0.3404      0.064      5.292      0.000       0.214       0.467
Finance Insurance And Real Estate                                  -0.1784      0.127     -1.405      0.160      -0.427       0.070
Manufacturing                                                      -0.0182      0.052     -0.351      0.726      -0.120       0.083
Mining                                                             -0.0450      0.048     -0.947      0.344      -0.138       0.048
Retail Trade                                                        0.1256      0.074      1.699      0.089      -0.019       0.270
Services                                                           -0.2630      0.085     -3.105      0.002      -0.429      -0.097
Transportation Communications Electric Gas And Sanitary Service    -0.1285      0.058     -2.215      0.027      -0.242      -0.015
==============================================================================
Omnibus:                    18757.262   Durbin-Watson:                   1.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           159948.347
Skew:                           0.533   Prob(JB):                         0.00
Kurtosis:                       8.657   Cond. No.                         51.5
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [8]:
#generate categorical variable for probit/logit analysis
data_w_dummies['paydividend'] = data_w_dummies['dps']>0
data_w_dummies['paydividend']	#purely for describing data
data_w_dummies['paydividend'].mean()	#purely for describing data
data_w_dummies['paydividend'] = data_w_dummies['paydividend'].astype(int)	#formatting the data for estimation models

In [9]:
#try using OLS anyway
result = sm.OLS(data_w_dummies['paydividend'], sm.add_constant(data_w_dummies[['lnoperatingmargin', 'Agriculture Forestry And Fishing', 'Construction','Finance Insurance And Real Estate', 'Manufacturing', 'Mining','Retail Trade', 'Services','Transportation Communications Electric Gas And Sanitary Service']]), missing='drop').fit(cov_type='cluster', cov_kwds={'groups': data_w_dummies['siccode']})
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            paydividend   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.104
Method:                 Least Squares   F-statistic:                     20.87
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           1.62e-35
Time:                        08:47:17   Log-Likelihood:                -77645.
No. Observations:              115861   AIC:                         1.553e+05
Df Residuals:                  115851   BIC:                         1.554e+05
Df Model:                           9                                         
Covariance Type:              cluster                                         
===================================================================================================================================
                                                                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                               0.5519      0.019     29.824      0.000       0.516       0.588
lnoperatingmargin                                                   0.0716      0.009      8.018      0.000       0.054       0.089
Agriculture Forestry And Fishing                                    0.0226      0.117      0.192      0.847      -0.208       0.253
Construction                                                       -0.0074      0.057     -0.129      0.897      -0.120       0.105
Finance Insurance And Real Estate                                   0.2932      0.032      9.187      0.000       0.231       0.356
Manufacturing                                                       0.0932      0.028      3.334      0.001       0.038       0.148
Mining                                                              0.0665      0.071      0.933      0.351      -0.073       0.206
Retail Trade                                                        0.1043      0.038      2.754      0.006       0.030       0.178
Services                                                           -0.0502      0.033     -1.519      0.129      -0.115       0.015
Transportation Communications Electric Gas And Sanitary Service     0.2445      0.046      5.288      0.000       0.154       0.335
==============================================================================
Omnibus:                   584262.775   Durbin-Watson:                   0.621
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12110.797
Skew:                          -0.030   Prob(JB):                         0.00
Kurtosis:                       1.417   Cond. No.                         51.5
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [10]:
#use logit instead
result = smdiscrete.Logit(data_w_dummies['paydividend'], sm.add_constant(data_w_dummies[['lnoperatingmargin', 'Agriculture Forestry And Fishing', 'Construction','Finance Insurance And Real Estate', 'Manufacturing', 'Mining','Retail Trade', 'Services','Transportation Communications Electric Gas And Sanitary Service']]), missing='drop').fit(cov_type='cluster', cov_kwds={'groups': data_w_dummies['siccode']})
result.summary()

Optimization terminated successfully.
         Current function value: 0.638094
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            paydividend   No. Observations:               115861
Model:                          Logit   Df Residuals:                   115851
Method:                           MLE   Df Model:                            9
Date:                Sun, 11 Oct 2020   Pseudo R-squ.:                 0.07838
Time:                        08:47:28   Log-Likelihood:                -73930.
converged:                       True   LL-Null:                       -80218.
Covariance Type:              cluster   LLR p-value:                     0.000
===================================================================================================================================
                                                                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                               0.2637      0.088      2.989      0.003       0.091       0.437
lnoperatingmargin                                                   0.3209      0.044      7.249      0.000       0.234       0.408
Agriculture Forestry And Fishing                                    0.0979      0.498      0.196      0.844      -0.879       1.075
Construction                                                       -0.0196      0.258     -0.076      0.940      -0.525       0.486
Finance Insurance And Real Estate                                   1.2749      0.170      7.494      0.000       0.941       1.608
Manufacturing                                                       0.3926      0.116      3.373      0.001       0.164       0.621
Mining                                                              0.2640      0.293      0.900      0.368      -0.311       0.839
Retail Trade                                                        0.4485      0.156      2.868      0.004       0.142       0.755
Services                                                           -0.2165      0.148     -1.460      0.144      -0.507       0.074
Transportation Communications Electric Gas And Sanitary Service     1.0202      0.208      4.903      0.000       0.612       1.428
===================================================================================================================================
"""

==end==